In [1]:
#Alternative : 
#import files
#from local_navigation import prox
#import class
#from thymioV2 import thymio_robot
#import librairies
import time
import numpy as np
import math 


In [2]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node() #_ = protected #__ = private = shouldn't access node outside of the class
await node.lock()

Node 95b5d775-2dea-417c-9b71-d572afe275af

In [33]:
#Thymio control
def motors(l_speed=500, r_speed=500):
    return {
        "motor.left.target": [l_speed],
        "motor.right.target": [r_speed],
    }

async def forward(motor_speed):
    await node.set_variables(motors(motor_speed,motor_speed))

async def motorset(motor_speed_left,motor_speed_right):
    await node.set_variables(motors(motor_speed_left,motor_speed_right))

async def rotate(theta, motor_speed): #theta is in radians
    if not (theta < 0.001 ):
        direction_rot=(theta>=0)-(theta<0)
        await node.set_variables(motors(motor_speed*direction_rot, -motor_speed*direction_rot))
        # wait time to get theta 1.44 is the factor to correct
        time=(theta)*100/motor_speed*1.4
        print(time)
        await(client.sleep(time))
    # stop the robot
        await node.set_variables(motors(0, 0))

async def stop_motor():
    await node.set_variables(motors(0,0))


async def get_proximity_values():
    # Wait for the Thymio node
    node = await client.wait_for_node()
    # Wait for the proximity sensor variables
    await node.wait_for_variables({"prox.horizontal"})
    # Get the proximity values : v: Stands for "variables" and is used to access the cached variable values.
    proximity_values = node.v.prox.horizontal
    # Return the value of the front proximity sensor (index 2)
    return proximity_values[0:5]

async def move_forward(dist, speed): 
    
    await node.set_variables(motors(speed, speed))
    # wait time to get theta 1.44 is the factor to correct
    speed2 = 3.5*speed
    time= 100*dist/speed2
    await(client.sleep(time))
    # stop the robot
    await node.set_variables(motors(0, 0))

# Run the asynchronous function and print the result
proximity_values = await get_proximity_values()
print(f"Front Proximity Sensor Value: {proximity_values}")
#await rotate(np.pi,100)


Front Proximity Sensor Value: [0, 0, 0, 0, 0]


In [49]:
await rotate(np.pi, 100)

In [24]:
await move_forward(10, 100)

In [14]:
node.unlock()

<generator object ClientAsyncNode.unlock at 0x000001B3F3CCBBC0>

In [7]:
def convert_to_idx(position, size_cell):
    idx =[0,0]
    idx[0] = int(np.floor(position[0]/size_cell))
    idx[1] = int(np.floor(position[1]/size_cell))
    return idx

In [37]:
idx = convert_to_idx([30.45, 61.2],2)
idx

[15, 30]

In [38]:
def test_if_goal(goal, position_robot):
    # Displacement vec
    if convert_to_idx(position_robot,2) == goal:
        return True
    else:
        return False

In [4]:
def compute_angle(x1,x2 ):
    """Calculer l'angle entre la direction actuelle et la direction vers l'arrivée."""
    angle_rad = math.atan2(x2[1] - x1[1], x2[0] - x1[0])
    #angle_deg = math.degrees(angle_rad)
    return angle_rad

In [43]:
compute_angle([1,2],[4,5])

0.7853981633974483

In [44]:
np.pi/4

0.7853981633974483

In [5]:
async def move_to_goal (x_est, goal, speed): 
   
    position_robot = [x_est[0],x_est[1]]
    angle_robot = x_est[2]
    pos_idx = convert_to_idx(position_robot,2)
    angle_goal = compute_angle(pos_idx, goal)
    print(angle_goal-angle_robot)

    await rotate(angle_robot-angle_goal, speed)
    await forward(100)
    
    

        

    

In [34]:
await move_to_goal([2,4, 3*np.pi/4], [4,5], 100)

-1.5707963267948966
2.199114857512855


In [35]:
await stop_motor()